In [ ]:
# Визуализации

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
plt.style.use("bmh")
plt.rcParams["axes.titlesize"] = "large"

In [ ]:
# Общеe

import os
import tqdm
import pathlib
import numpy as np
import pandas as pd

tqdm.tqdm.pandas()
pd.set_option('display.max_columns', None)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from catboost import CatBoostRegressor

<div class="alert alert-block alert-warning">
Не забудьте заменить <tt>DATA_DIR</tt> на имя локальной директории, в которую сохранили файл.
</div>

In [ ]:
DATA_DIR = pathlib.Path("")
RS = 3984765

# Загрузка данных

Данные хранятся в формате Parquet: читается быстрее, места занимает меньше.

In [ ]:
train = pd.read_parquet(DATA_DIR.joinpath("train.parquet"))
train.head()

In [ ]:
train.corr()

In [ ]:
for_show = train.iloc[:, [0, 1, 2, 3,4,  -2, -3]]

In [ ]:
for_show.shape

In [ ]:
for_show = train.iloc[:, [4, 5, 6, 7, 8,  -2, -3]]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.pairplot(for_show)

In [ ]:
import plotly.express as px
fig = px.scatter_matrix(for_show, color=for_show['feature4'])
fig.show()

In [ ]:
for_show = train.iloc[:, [4, 9, 10, 11, 12,  -2, -3]]

fig = px.scatter_matrix(train.iloc[:, [9, 10, 11, 12,  -2, -3]], dimensions=train.iloc[:, [9, 10, 11, 12,  -2, -3]], color=for_show['feature4'])
fig.show()

In [ ]:
for_show = train.iloc[:, [4, 13, 14, 15, 16,  -2, -3]]

fig = px.scatter_matrix(for_show, color=for_show['feature4'])
fig.show()

In [ ]:
for_show = train.iloc[:, [4, 17, 18, 19, 20,  -2, -3]]

fig = px.scatter_matrix(for_show, color=for_show['feature4'])
fig.show()

In [ ]:
for_show = train.iloc[:, [4, 21, 22, 23, 24,  -2, -3]]

fig = px.scatter_matrix(for_show, color=for_show['feature4'])
fig.show()

In [ ]:
for_show

В данных есть один категориальный признак: `feature4`. Он может принимать значения `gas1` и `gas2`. Таргета два: `target0` и `target1`.

In [ ]:
CAT = ["feature4"]
TARGETS = ["target0", "target1"]
FTS = train.filter(like="feature").columns.difference(CAT)

Проверяем, что с пропусками:

In [ ]:
train.isnull().any().mean()

Их нет! Вот это поворот! Посмотрим на таргеты:

In [ ]:
plt.figure(figsize=(6,6))

train[TARGETS].plot(kind="hist", range=(0, 100), bins=20, alpha=0.6, ax=plt.gca())
plt.xlabel("target");

Категориальный признак меняем на числовой:

In [ ]:
train["gas"] = 0
train.loc[train.feature4=="gas2", "gas"] = 1

In [ ]:
FTS = FTS.union(["gas"])

# Простая модель

Построим простую модель со случайным разбиением на тренировочное и валидационное множества. Таргеты будем моделировать отдельно.

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(train[FTS], train[TARGETS], train_size=0.5, random_state=RS)

In [ ]:
models = {}
tr_preds = []
val_preds = []

for tg in TARGETS:
    print(f"{tg}", "=" * 10)
    cb_model = CatBoostRegressor(max_depth=4, iterations=5000,
                                 early_stopping_rounds=20, objective="MAPE", verbose=200,
                                 random_state=RS)
    cb_model.fit(X_tr, y_tr[tg], eval_set=(X_val, y_val[tg]))

    tr_preds.append(cb_model.predict(X_tr))
    val_preds.append(cb_model.predict(X_val))

    models[tg] = cb_model

Собираем предсказания вместе:

In [ ]:
tr_preds = np.column_stack(tr_preds)
val_preds = np.column_stack(val_preds)

In [ ]:
tr_preds = np.clip(tr_preds, 0, 100)
val_preds = np.clip(val_preds, 0, 100)

Посчитаем метрики (так они считаются и для загрузок на платформе):

In [ ]:
print(f"MAPE (train): {mean_absolute_percentage_error(y_tr, tr_preds) * 100:.3f} %")
print(f"MAPE (val): {mean_absolute_percentage_error(y_val, val_preds) * 100:.3f} %")

Сохраним модели:

In [ ]:
for target, model in models.items():
    model.save_model(DATA_DIR.joinpath(f"{target}-cb-v1.cbm"))

Эти модели (в зависимости от `RS`) должны давать на публичном лидерборде `~10%`. Как сделать так, чтобы локальная валидация сходилась с лидербордом: вот в чем вопрос!